# 목표 : 수익률 PEIS점수테이블에 붙이기 

# 흐름 (괄호는 코스닥만)
1. 읽어들이기 
2. 테이블 다중인덱스로 만들기 --> 편하게 하기위해서 --> PEIS 랑 똑같은 데이터 프레임형태
2. 목표주가, 당시주가, 수정주가, 애널리스트등급, 시가총액 일자로 세워서 붙이기
3. 수정주가 기준 수익률 계산해주기
4. fillna 99 해서 수익률 없는거 99해주기 --> 나중에 지우기 쉽게하기위해 임의로 박아준거임
* (코스닥) - "스팩" 데이터 제거하기 
5. 목표주가랑 현재주가 기준 컨센 구하기 
6. PEIS랑 주가데이터랑 결합해서 저장

In [1]:
import pandas as pd
import numpy as np
import warnings
from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 파일불러들이기 붙여질 파일(PEIS) + 붙일 파일(price)
*# 아시아나항공이랑 한섬은 stock 데이터에서 일부분이 빠져서 버림 이거 찾는데 1시간 반걸림 ㅋㅋ ~~~~~

In [2]:
PEIS_table = pd.read_excel("Data/PEISdata.xlsx",sheet_name = "kospi_peis")
PROFIT_table = pd.read_excel("Data/PEISdata.xlsx",sheet_name = "kospi_stock")

## 1.기업별로 Match 시키기 위해서 PROFIT쪽에서 기업이름하고 티커 맞추기 --> 2번 상장경우가 존재


In [3]:
PROFIT_com_name = PROFIT_table['Symbol Name'].tolist()
PROFIT_com_symbol = PROFIT_table['Symbol'].tolist()
PROFIT_namesymbol = []
for i in range(len(PROFIT_com_name)):
    PROFIT_namesymbol.append([PROFIT_com_name[i],PROFIT_com_symbol[i]]) # 사실 여기서 8로 나눴으면 좋았을텐데 속도 적으로 

# 2. 양쪽에 불균형하게 존재하는 데이터제거 (상폐 때문인 듯) 
## 원컬럼으로 세웠을때 데이터 맞출려면 필수적

In [4]:
# 그냥 뽑을때부터 데이터가 서로 불균형하게 뽑히는 경우가 존재 --> 어쩔 수 없이 제거 
delistcom = set(PEIS_table['Name'].unique())-set(PROFIT_table['Symbol Name'].unique())
dropdata(PEIS_table, 'Name', delistcom)
Symbolindexcom = set(PEIS_table['Symbol'].unique())-set(PROFIT_table['Symbol'].unique())
dropdata(PEIS_table, 'Symbol', Symbolindexcom)

# 데이터 전처리 
# 3. 주가랑 주가상승률, 애널리스트등급 행렬로 일자로 만들기

In [5]:
PROFIT_table = MakeLookGood(PROFIT_table, years = list(range(8,22))) # 새로 뽑으면 10 -> 8
PROFIT_table = PROFIT_table.fillna(0)
Onetable = MakeOnetable(PROFIT_table, ['수정주가(원)','투자의견점수 (E3)(포인트)','시가총액 (티커-상장예정주식수 포함)(백만원)','종가(원)','적정주가 (E3)(원)','PER(배)','PBR(배)','베타 (D,1Yr)'], ['ADJprice', 'Consensus','시가총액','Endprice','Targetprice','PER','PBR','BETA'])
Onetable['deltaprice'] = Caldeltaprice(Onetable['ADJprice'])
print(Onetable.shape, PEIS_table.shape)

(16282, 9) (16282, 23)


# 4.PEIS_table 기업별로 나누기 
# +
# 5. 기업별로 PEIS_STOCK data 맞추기_전체 합치기

In [6]:
namebasedfs = namebaseDFs(PEIS_table)
PEISSTOCK_com_base = concatPEIS_STOCK(namebasedfs, PROFIT_namesymbol, Onetable)
Matchcompanybase = pd.concat(PEISSTOCK_com_base)
Matchcompanybase['Targetprice'].fillna(-1,inplace = True) # 카테고리화 이전에 하긴 해야하네
Matchcompanybase.reset_index(drop=True,inplace = True)
Matchcompanybase.to_excel('temp/kospi/Matchcompany.xlsx')

# 6 컨센 점수 가공 (제공 & 목표,종가 조합)

In [7]:
consen_cut = pd.cut(Matchcompanybase['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Matchcompanybase['consen_cut'] = consen_cut
CompuConsen(Matchcompanybase)

# 7. 스팩 제거 

In [8]:
names = Matchcompanybase['Name']
specindex = returnspecindex(names)
Matchcompanybase.drop(specindex, inplace = True)
Matchcompanybase.reset_index(drop = True, inplace = True)

# 8. 저장하기

In [9]:
Matchcompanybase.to_excel("Data/kospi/KospiPEISData&PriceData.xlsx",index= False)

#

#

#


#

#

#

#

#

#


#

# 코스닥 똑같이 만들기
* 코스닥도 나중에 코스피랑 똑같은 전처리 과정에서 문제 없는지 확인해야함
* 코스닥은 디엠티랑 한프 날림

# 0. 파일불러들이기 붙여질 파일(PEIS) + 붙일 파일(price)
*# 아시아나항공이랑 한섬은 stock 데이터에서 일부분이 빠져서 버림 이거 찾는데 1시간 반걸림 ㅋㅋ ~~~~~

In [10]:
PEIS_table_kosdaq = pd.read_excel("Data/PEISdata.xlsx",sheet_name = "kosdaq_peis")
PROFIT_table_kosdaq = pd.read_excel("Data/PEISdata.xlsx",sheet_name = "kosdaq_stock")

## 1.기업별로 Match 시키기 위해서 PROFIT쪽에서 기업이름하고 티커 맞추기 --> 2번 상장경우가 존재


In [11]:
PROFIT_com_name_kosdaq = PROFIT_table_kosdaq['Symbol Name'].tolist()
PROFIT_com_symbol_kosdaq = PROFIT_table_kosdaq['Symbol'].tolist()
PROFIT_namesymbol_kosdaq = []
for i in range(len(PROFIT_com_name_kosdaq)):
    PROFIT_namesymbol_kosdaq.append([PROFIT_com_name_kosdaq[i],PROFIT_com_symbol_kosdaq[i]]) # 사실 여기서 8로 나눴으면 좋았을텐데 속도 적으로 

# 2. 양쪽에 불균형하게 존재하는 데이터제거 (상폐 때문인 듯) 
## 원컬럼으로 세웠을때 데이터 맞출려면 필수적

In [12]:
# 그냥 뽑을때부터 데이터가 서로 불균형하게 뽑히는 경우가 존재 --> 어쩔 수 없이 제거 
delistcom_kosdaq = set(PEIS_table_kosdaq['Name'].unique())-set(PROFIT_table_kosdaq['Symbol Name'].unique())
dropdata(PEIS_table_kosdaq, 'Name', delistcom_kosdaq)
Symbolindexcom_kosdaq = set(PEIS_table_kosdaq['Symbol'].unique())-set(PROFIT_table_kosdaq['Symbol'].unique())
dropdata(PEIS_table_kosdaq, 'Symbol', Symbolindexcom_kosdaq)

# 데이터 전처리 
# 3. 주가랑 주가상승률, 애널리스트등급 행렬로 일자로 만들기

In [13]:
PROFIT_table_kosdaq = MakeLookGood(PROFIT_table_kosdaq, years = list(range(8,22))) # 새로 뽑으면 10 -> 8
PROFIT_table_kosdaq = PROFIT_table_kosdaq.fillna(0)
Onetable_kosdaq = MakeOnetable(PROFIT_table_kosdaq, ['수정주가(원)','투자의견점수 (E3)(포인트)','시가총액 (티커-상장예정주식수 포함)(백만원)','종가(원)','적정주가 (E3)(원)','PER(배)','PBR(배)','베타 (D,1Yr)'], ['ADJprice', 'Consensus','시가총액','Endprice','Targetprice','PER','PBR','BETA'])
Onetable_kosdaq['deltaprice'] = Caldeltaprice(Onetable_kosdaq['ADJprice'])

# 4.PEIS_table 기업별로 나누기 
# +
# 5. 기업별로 PEIS_STOCK data 맞추기_전체 합치기

In [14]:
namebasedfs_kosdaq = namebaseDFs(PEIS_table_kosdaq)
PEISSTOCK_com_base_kosdaq = concatPEIS_STOCK(namebasedfs_kosdaq, PROFIT_namesymbol_kosdaq, Onetable_kosdaq)
Matchcompanybase_kosdaq = pd.concat(PEISSTOCK_com_base_kosdaq)
Matchcompanybase_kosdaq['Targetprice'].fillna(-1,inplace = True)
Matchcompanybase_kosdaq.reset_index(drop=True,inplace = True)
Matchcompanybase_kosdaq.to_excel('temp/kosdaq/Matchcompany.xlsx')

# 6 컨센 점수 가공 (제공 & 목표,종가 조합)

In [15]:
consen_cut = pd.cut(Matchcompanybase_kosdaq['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Matchcompanybase_kosdaq['consen_cut'] = consen_cut
CompuConsen(Matchcompanybase_kosdaq)

# 7. 스팩 제거 

In [16]:
names = Matchcompanybase_kosdaq['Name']
specindex = returnspecindex(names)
Matchcompanybase_kosdaq.drop(specindex, inplace = True)
Matchcompanybase_kosdaq.reset_index(drop = True, inplace = True)

# 8. 저장하기

In [17]:
Matchcompanybase_kosdaq.to_excel("Data/kosdaq/KosdaqPEISData&PriceData.xlsx",index= False)